<a href="https://colab.research.google.com/github/Me-RockingAbhi/My-Projects/blob/master/E_Comm_Price_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
import numpy as np
import pandas as pd
data1=pd.read_csv("/content/gdrive/My Drive/E-com Train.csv")
data2=pd.read_csv("/content/gdrive/My Drive/E-com Test.csv")
print(data1.shape)
print(data2.shape)
ytr=data1['Selling_Price']
data1=data1.drop("Selling_Price",axis=1)

(2452, 8)
(1051, 7)


In [10]:
data2.isnull().sum()

Product          0
Product_Brand    0
Item_Category    0
Subcategory_1    0
Subcategory_2    0
Item_Rating      0
Date             0
dtype: int64

In [0]:
data=data1.append(data2)
data.shape
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['Product_Brand']=le.fit_transform(data['Product_Brand'])

In [12]:
print(data.describe)
print(data.shape)
data.isnull().sum()

<bound method NDFrame.describe of       Product  Product_Brand  ... Item_Rating        Date
0      P-2610           1130  ...         4.3    2/3/2017
1      P-2453            877  ...         3.1    7/1/2015
2      P-6802            369  ...         3.5   1/12/2019
3      P-4452            877  ...         4.0  12/12/2014
4      P-8454            877  ...         4.3  12/12/2013
...       ...            ...  ...         ...         ...
1046   P-9758            669  ...         2.0   28/2/2014
1047  P-11898            936  ...         4.7   1/12/2019
1048    P-904            160  ...         3.5   1/12/2014
1049   P-1714           1017  ...         2.4   23/6/2013
1050    P-620            160  ...         3.1   1/12/2012

[3503 rows x 7 columns]>
(3503, 7)


Product          0
Product_Brand    0
Item_Category    0
Subcategory_1    0
Subcategory_2    0
Item_Rating      0
Date             0
dtype: int64

In [13]:
data.dtypes

Product           object
Product_Brand      int64
Item_Category     object
Subcategory_1     object
Subcategory_2     object
Item_Rating      float64
Date              object
dtype: object

In [14]:
for col in data.columns:
  print(col,data[col].nunique())

Product 3503
Product_Brand 1313
Item_Category 95
Subcategory_1 142
Subcategory_2 395
Item_Rating 41
Date 563


In [0]:
import datetime
data['year'] = pd.DatetimeIndex(data['Date']).year
data['month'] = pd.DatetimeIndex(data['Date']).month

In [0]:
day=[]
for i in data['Date']:
  name=i.split("/")[0]
  day.append(int(name))
data['day']=day

In [0]:
data['Item_Category']=data['Item_Category']+" "+data['Subcategory_1']+" "+data['Subcategory_2']

In [18]:
data.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,year,month,day
0,P-2610,1130,bags wallets belts bags hand bags,bags,hand bags,4.3,2/3/2017,2017,2,2
1,P-2453,877,clothing women s clothing western wear,women s clothing,western wear,3.1,7/1/2015,2015,7,7
2,P-6802,369,home decor festive needs showpieces ethnic,showpieces,ethnic,3.5,1/12/2019,2019,1,1
3,P-4452,877,beauty and personal care eye care h2o plus eye...,eye care,h2o plus eye care,4.0,12/12/2014,2014,12,12
4,P-8454,877,clothing men s clothing t shirts,men s clothing,t shirts,4.3,12/12/2013,2013,12,12


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vec = TfidfVectorizer(stop_words='english',max_features=50000)
text= tfidf_vec.fit_transform(data['Item_Category'].values.tolist())
text= pd.DataFrame(text.toarray(), columns=tfidf_vec.get_feature_names())

In [20]:
print(text.shape)
print(data.shape)
data=data.drop("Item_Category",axis=1)

(3503, 869)
(3503, 10)


In [0]:
data=data.drop("Subcategory_1",axis=1)
data=data.drop("Subcategory_2",axis=1)
data=data.drop("Date",axis=1)
data=data.drop("Product",axis=1)

In [0]:
train_d=text[:2452]
test_d=text[2452:]
data_tr=data[:2452]
data_te=data[2452:]
data_tr=pd.concat([train_d,data_tr],axis=1)

In [23]:
test_d.isnull().sum()

01          0
1000m       0
122         0
13          0
216456      0
           ..
zikrak      0
zir         0
zircon      0
zirconia    0
zoop        0
Length: 869, dtype: int64

In [24]:
test_d.reset_index(drop=True, inplace=True)
data_te=pd.concat([test_d,data_te],axis=1)
test_d.isnull().sum()

01          0
1000m       0
122         0
13          0
216456      0
           ..
zikrak      0
zir         0
zircon      0
zirconia    0
zoop        0
Length: 869, dtype: int64

In [25]:
data_tr.head()

,01,1000m,122,13,216456,216552,218,22,30,34,3514,36,37,40,4ths,51,53,54,56m,80,abhushan,academic,accessories,action,activity,adaa,adapters,adidas,adraxx,adventure,aimedu,air,akup,aligatorr,alloy,alpine,amantran,amethyst,amita,amy,...,view,vinson,viscose,vishudh,vk,wall,wallets,wallpapers,wash,watch,watches,water,wayfarer,weapons,wear,wedges,western,white,wine,winter,wired,wishkey,women,wow,wrist,yellow,yo,you2deal,young,zevrr,zikrak,zir,zircon,zirconia,zoop,Product_Brand,Item_Rating,year,month,day
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.286506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1130,4.3,2017,2,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.398015,0.0,0.492902,0.0,0.0,0.0,0.0,0.0,0.367128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,877,3.1,2015,7,7
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,369,3.5,2019,1,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,877,4.0,2014,12,12
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,877,4.3,2013,12,12


In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
gbrt=GradientBoostingRegressor(n_estimators=600,max_depth=3)
param_grid = { 
    'n_estimators': [400,450,500,550,650, 700,800],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth':[3,4,5,6]
}
CV_gbrt = GridSearchCV(estimator=gbrt, param_grid=param_grid, cv= 4)
CV_gbrt.fit(data_tr,ytr)
print(CV_gbrt.best_params_)

{'max_depth': 4, 'max_features': 'auto', 'n_estimators': 550}


In [0]:
from sklearn.feature_selection import SelectFromModel
gbrt=GradientBoostingRegressor(n_estimators=550,max_depth=4,max_features='auto')
select_model = SelectFromModel(gbrt)
fit_d= select_model.fit(data_tr, ytr)
model_features_t = fit_d.transform(data_tr)
model_features_te = fit_d.transform(data_te)

In [0]:
data_te.isnull().sum()

In [0]:
gbrt.fit(data_tr,ytr)
ans=gbrt.predict(data_te)
new=pd.DataFrame()
new['Selling_Price']=ans
new.to_excel("solut.xlsx",index=False)